## 1. 필요 모듈 설치

In [1]:
!pip install -q pyperclip
!pip install -q webdriver-manager
!pip install -q chromedriver-autoinstaller

## 2. 모듈 임포트

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

import pyperclip
import time
import csv
import pandas as pd

from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_autoinstaller  # setup chrome options

## 3. 크롬 드라이버 옵션

In [3]:
options = webdriver.ChromeOptions()
# options.add_argument('--headless') # ensure GUI is off
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
options.add_argument('--disable-gpu')
# options.add_argument('--disable-javascript')
options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

'C:\\Users\\etukd\\anaconda3\\Lib\\site-packages\\chromedriver_autoinstaller\\125\\chromedriver.exe'

## 4. 네이버 시리즈온 크롤링

### 4-1. URL 설정

In [4]:
# base_url = "https://serieson.naver.com/v3/movie/products/action?sortType=UPDATE_DESC&price=all" # 액션
# genre = 'action'

# base_url = "https://serieson.naver.com/v3/movie/products/horror?sortType=UPDATE_DESC&price=all" # 공포/스릴러
# genre = 'horror'

base_url = "https://serieson.naver.com/v3/movie/products/drama?sortType=UPDATE_DESC&price=all" # 드라마
genre = 'drama'

In [35]:
try:
    saved_df = pd.read_csv(f'{genre}_movie_data.csv')
    start_num = saved_df.shape[0] 
    start_num -= 1 # saved_data의 마지막과 save_data의 처음을 겹치게 하기 위해
except:
    start_num = 1
finally:
    print(start_num)

12301


### 4-2. 네이버 시리즈온 로그인

In [40]:
driver = webdriver.Chrome(options=options)
driver.get(base_url)

time.sleep(3)

ID = 'etukdh6433' #본인ID
PW = 'tuticus643#' #본인PW

try:
    login_btn = driver.find_element(By.CSS_SELECTOR, '#gnb_login_button')
    login_btn.click()
    
    #driver.get("https://nid.naver.com/nidlogin.login")
    time.sleep(3)
    
    id = driver.find_element(By.CSS_SELECTOR, "#id")
    id.click()
    pyperclip.copy(ID)
    actions = ActionChains(driver)
    actions.key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)
    # id.send_keys(ID)
    
    pw = driver.find_element(By.CSS_SELECTOR, "#pw")
    pw.click()
    pyperclip.copy(PW)
    actions = ActionChains(driver)
    actions.key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)
    # pw.send_keys(PW)
    
    btn_login = driver.find_element(By.CSS_SELECTOR, "#log\.login")
    btn_login.click()
    
except Exception as e:
    print(f'Login Error Occurred: {e}')

finally:
    time.sleep(5)

In [41]:
# 영화 리스트 수 추출
h3_text = driver.find_element(By.CSS_SELECTOR, '#content > div.ListPage_category_wrap__L1R0q > div.ListCollectionTitleBar_wrapper__xhXIY > div.CollectionTitleBar_root__NRk1G > h3').text
start_idx = h3_text.find('(')
end_idx = h3_text.find(')')
end_num = int(h3_text[start_idx + 1:end_idx].replace(',',''))+1

# 업데이트 존재 시 수정 필요
# end_num = 5010

print(end_num)

13394


### 4-3. 네이버 시리즈온 더보기

In [42]:
# 더보기 클릭
while True:
    try:
        more_button = driver.find_element(By.CSS_SELECTOR, 'button.ViewMoreButton_view_more_button__eGoJu')
        more_button.click()
        time.sleep(2)
    except:
        break

### 4-4. 네이버 시리즈온 데이터 크롤링

In [43]:
# 영화 데이터 크롤링
movie_data = {
    'title': [],
    'rating': [],
    'year': [],
    'runtime': [],
    'age': [],
    'isDRM': [],
    'director': [],
    'actors': [],
    'type': [],
    'price': [],
    'version': [],
    'period': [],
    'like': [],
    'comment': []
}

try:
    for i in range(start_num, end_num):
        movie = driver.find_element(By.CSS_SELECTOR, f'#content > div.ListPage_category_wrap__L1R0q > ul > li:nth-child({i})')
        # 영화 세부 페이지로 이동
        movie.click()
        time.sleep(1)
        
        title = driver.find_element(By.CSS_SELECTOR, 'div.title_area > div.title > strong').text
        # print(title)

        rating, year, runtime, age, isDRM = 0, None, None, None, None
        info = driver.find_elements(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div[2]//span') # rating, year, runtime, age, isDRM
        for i in info:
            target = i.text
            
            if not target.endswith('전용'):
                if target.endswith('관람가') or target.endswith('관람불가'):
                    age = target
                elif target.endswith('분'):
                    runtime = target
                elif target == 'DRM':
                    isDRM = target
                elif float(target)<11:
                    rating = target
                else:
                    year = target
        # print(rating, year, runtime, age, isDRM)

        try:
            description = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/ul/li[1]/div[1]/div/a/span').text
        except:
            description = None
            pass
        # print(description)

        try:
            actors = ''
            cast = driver.find_elements(By.XPATH, '//*[@id="content"]/div[2]/ul/li[1]/div[2]/div//span')
            for c in cast:
                actors += c.text
            if actors == '':
                actors = None
        except:
            actors = None
            pass
        # print(actors)

        purchase = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[2]').text.split()
        # purchase = ['단건', ('대여'or'구매'), (금액)]
        if purchase:
            type, price = purchase[1:]
        else:
            type, price = '무료', 0
        # print(type, price)


        ## 추가 항목
        is_sub = driver.find_element(By.CSS_SELECTOR, '#content > div.end_contents_info > div > div.title_area > div.title > span.bullet.subtitle').get_attribute('style')
        is_dub = driver.find_element(By.CSS_SELECTOR, '#content > div.end_contents_info > div > div.title_area > div.title > span.bullet.dubbing').get_attribute('style')
        if is_sub and not is_dub:
            version = '더빙'
        elif is_dub and not is_sub:
            version = '자막'
        else:
            version = None
        # print(version)

        # '이용기간 ' 제외
        period = driver.find_element(By.CSS_SELECTOR, '#content > div.end_contents_info > ul > li.info_list.type_price > div > div:nth-child(2) > div > span:nth-child(2)').text.replace('이용기간 ', '')
        # print(period)
        
        like = driver.find_element(By.CSS_SELECTOR, '#likeit > a > em.u_cnt._count').text
        if not like:
            like = 0
        # print(like)

        comment = driver.find_element(By.CSS_SELECTOR, '#content > div.end_contents_info > div > ul > li:nth-child(3) > button').text
        if comment == '감상평':
            comment = 0
        # print(comment)
        
        # data 입력
        movie_data['title'].append(title)
        movie_data['rating'].append(rating)
        movie_data['year'].append(year)
        movie_data['runtime'].append(runtime)
        movie_data['age'].append(age)
        movie_data['isDRM'].append(isDRM)
        movie_data['director'].append(description)
        movie_data['actors'].append(actors)
        movie_data['type'].append(type)
        movie_data['price'].append(price)
        movie_data['version'].append(version)
        movie_data['period'].append(period)
        movie_data['like'].append(like)
        movie_data['comment'].append(comment)
        
        # 영화 리스트 페이지로 이동
        driver.back()
        time.sleep(2)
        # break
        
    print(f'입력된 영화 리스트 수 : {len(movie_data["title"])}')
    
except Exception as e:
    print(f'Crawl Data Error Occurred: {e}')

finally:
    driver.close()


Crawl Data Error Occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.title_area > div.title > strong"}
  (Session info: chrome=125.0.6422.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00B3B8E3+45827]
	(No symbol) [0x00ACDCC4]
	(No symbol) [0x009C150F]
	(No symbol) [0x00A020BC]
	(No symbol) [0x00A0216B]
	(No symbol) [0x00A3E0F2]
	(No symbol) [0x00A22E44]
	(No symbol) [0x00A3C034]
	(No symbol) [0x00A22B96]
	(No symbol) [0x009F6998]
	(No symbol) [0x009F751D]
	GetHandleVerifier [0x00DF4513+2899763]
	GetHandleVerifier [0x00E4793D+3240797]
	GetHandleVerifier [0x00BC13B4+593364]
	GetHandleVerifier [0x00BC82DC+621820]
	(No symbol) [0x00AD70A4]
	(No symbol) [0x00AD37A8]
	(No symbol) [0x00AD3947]
	(No symbol) [0x00AC59FE]
	BaseThreadInitThunk [0x75157BA9+25]
	RtlInitializeExceptionChain [0x772CC10B+107]
	RtlC

## 5. 데이터 확인

In [44]:
saved_df.tail()

,title,rating,year,runtime,age,isDRM,director,actors,type,price,version,period,like,comment,actor,cost
12297,8 마일,8.82,2003.0,110분,15세 관람가,DRM,커티스 핸슨,"에미넴,킴 베이싱어,브리트니 머피,안소니 마키,메키 파이퍼,에반 존스,오마 벤슨 밀...",대여,"1,200",NaN,2일,55,10,NaN,NaN
12298,8 마일,8.82,2003.0,110분,15세 관람가,DRM,커티스 핸슨,"에미넴,킴 베이싱어,브리트니 머피,안소니 마키,메키 파이퍼,에반 존스,오마 벤슨 밀...",구매,"5,000",NaN,5년,27,5,NaN,NaN
12299,21 그램,8.20,2004.0,124분,청소년 관람불가,DRM,알레한드로 곤잘레스 이냐리투,"숀 펜,베니시오 델 토로,나오미 왓츠,샤롯 갱스부르,멜리사 레오,에디 마산,클리어 ...",대여,"1,200",NaN,2일,18,0,NaN,NaN
12300,21 그램,8.20,2004.0,124분,청소년 관람불가,DRM,알레한드로 곤잘레스 이냐리투,"숀 펜,베니시오 델 토로,나오미 왓츠,샤롯 갱스부르,멜리사 레오,에디 마산,클리어 ...",구매,"5,000",NaN,5년,13,0,NaN,NaN
12301,춤추는 나의 베아트리체,7.17,2007.0,112분,청소년 관람불가,DRM,안토니오 반데라스,"알베르토 아마릴라,마리아 루이즈,펠릭스 고메즈,라울 아레발로,마르타 니에토,마리오 ...",구매,"1,000",NaN,제한 없음,0,0,NaN,NaN


In [46]:
save_df = pd.DataFrame(movie_data)
save_df.head(30)

# save_df = save_df.iloc[4:]

,title,rating,year,runtime,age,isDRM,director,actors,type,price,version,period,like,comment
0,8 마일,8.82,2003,110분,15세 관람가,DRM,커티스 핸슨,"에미넴,킴 베이싱어,브리트니 머피,안소니 마키,메키 파이퍼,에반 존스,오마 벤슨 밀...",대여,"1,200",None,2일,55,10
1,8 마일,8.82,2003,110분,15세 관람가,DRM,커티스 핸슨,"에미넴,킴 베이싱어,브리트니 머피,안소니 마키,메키 파이퍼,에반 존스,오마 벤슨 밀...",구매,"5,000",None,5년,27,5
2,21 그램,8.2,2004,124분,청소년 관람불가,DRM,알레한드로 곤잘레스 이냐리투,"숀 펜,베니시오 델 토로,나오미 왓츠,샤롯 갱스부르,멜리사 레오,에디 마산,클리어 ...",대여,"1,200",None,2일,18,0
3,21 그램,8.2,2004,124분,청소년 관람불가,DRM,알레한드로 곤잘레스 이냐리투,"숀 펜,베니시오 델 토로,나오미 왓츠,샤롯 갱스부르,멜리사 레오,에디 마산,클리어 ...",구매,"5,000",None,5년,13,0
4,춤추는 나의 베아트리체,7.17,2007,112분,청소년 관람불가,DRM,안토니오 반데라스,"알베르토 아마릴라,마리아 루이즈,펠릭스 고메즈,라울 아레발로,마르타 니에토,마리오 ...",구매,"1,000",None,제한 없음,0,0
5,고 웨스트,10,None,183분,15세 관람가,DRM,안드레아스 린케,"프랜즈 딘다,세르게이 모야,프레더릭 라우,잉카 프리드리흐,칼 크란즈코우스키",구매,"1,500",None,제한 없음,0,0
6,"아이,애나",5.95,2013,91분,15세 관람가,DRM,바너비 사우스캠,"샤롯 램플링,가브리엘 번,헤일리 앳웰,랠프 브라운,에디 마산,조디 메이,호노 블래크...",구매,"6,500",None,제한 없음,4,0
7,고령화가족,8.25,2013,113분,15세 관람가,DRM,송해성,"박해일,윤제문,공효진,윤여정,진지희,예지원,김영재,유승목,박영서,이영진",구매,"6,050",None,제한 없음,16,6
8,고령화가족,8.25,2013,113분,15세 관람가,DRM,송해성,"박해일,윤제문,공효진,윤여정,진지희,예지원,김영재,유승목,박영서,이영진",대여,"1,540",None,7일,15,3
9,런던의 늑대인간,7,None,75분,12세 관람가,DRM,스튜어트 워커,"헨리 헐,워너 올랜드,발레리 홉슨,레스터 매튜스,로렌스 그랜트,스프링 바잉톤,클락 ...",구매,"1,000",None,제한 없음,2,0


In [47]:
save_df.describe()

,title,rating,year,runtime,age,isDRM,director,actors,type,price,version,period,like,comment
count,183,183,103,183,183,183,183,183,183,183,3,183,183,183
unique,165,115,26,95,4,1,139,166,2,17,2,4,60,23
top,8 마일,0,2013,100분,15세 관람가,DRM,구로사와 아키라,"에미넴,킴 베이싱어,브리트니 머피,안소니 마키,메키 파이퍼,에반 존스,오마 벤슨 밀...",구매,"1,000",자막,제한 없음,0,0
freq,2,8,36,7,78,183,13,2,166,62,2,147,18,103


In [49]:
result = pd.concat([saved_df,save_df.iloc[5:]], ignore_index=True)
# result.head()
# result.tail()
result.describe()

,title,rating,year,runtime,age,isDRM,director,actors,type,price,version,period,like,comment,actor,cost
count,12480,10196.0,7855.0,12480,12480,12477,12464,898,12466,909,699,12480,12480,12480,11039,11557
unique,8633,759.0,101.0,202,6,1,5587,771,2,26,2,5,456,118,7458,56
top,비스트,10.0,2023.0,101분,15세 관람가,DRM,홍상수,"에바 그린,설리반 스탭플턴,로드리고 산토로,레나 헤디,한스 매디슨,앤드류 티어난,아...",구매,"1,000",자막,제한 없음,0,0,"송강호,이선균,조여정,최우식,박소담,이정은,장혜진,박명훈,현승민,정현준","1,200"
freq,6,206.0,499.0,293,4684,12477,65,4,8534,210,420,6866,2588,8226,6,1320


## 6. 데이터 저장

In [50]:
result.to_csv(f'{genre}_movie_data.csv', index=False)